In [ ]:
!apt-get install unzip

In [ ]:
!pip install tokenizers matplotlib sklearn

In [ ]:
# в vast ai или в последних версия jupyter может не работать автозаполнение, установка этой либы и перезагрука кернела помогает
!pip install --upgrade jedi==0.17.2

In [5]:

import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
! wget -O opus.en-ru-train.en.txt http://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en

In [ ]:
! wget -O opus.en-ru-train.ru.txt http://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru

In [8]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru.txt', 'w')
f.write(text)
f.close()

In [9]:
en_sents = open('opus.en-ru-train.en.txt').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru.txt').read().lower().splitlines()

In [10]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

In [11]:
tokenizer_en = Tokenizer(BPE(), )
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[PAD]", "[CLS]", "[SEP]", "[UNK]", "[MASK]"], )
tokenizer_en.train(files=["opus.en-ru-train.en.txt"], trainer=trainer_en, )

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[PAD]", "[CLS]", "[SEP]", "[UNK]", "[MASK]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru.txt"], trainer=trainer_ru)

In [12]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [13]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [14]:
def encode(text, tokenizer, target=False):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]


In [15]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, target=True) for t in ru_sents]

In [16]:
# max_len_en = np.mean([len(x) for x in X_en])
# max_len_ru = np.mean([len(x) for x in X_ru])

In [17]:
# max_len_en, max_len_ru

In [18]:
# ограничимся длинной в 20 и 22 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 20, 22

In [19]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

0

In [20]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post')
X_ru_out = tf.keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post', )

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1, padding='post')

In [22]:
# миллион примеров 
X_en.shape, len(X_ru)

((1000000, 20), 1000000)

In [23]:
X_en_train, X_en_valid, X_ru_dec_train, X_ru_dec_valid, X_ru_out_train, X_ru_out_valid = train_test_split(X_en, 
                                                                                                      X_ru_dec, 
                                                                                                      X_ru_out, 
                                                                                                      test_size=0.05)

In [24]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [25]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [26]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [27]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [28]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [29]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [30]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [31]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [32]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [33]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [34]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [35]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

# def loss_function(real, pred):
#   mask = tf.math.logical_not(tf.math.equal(real, 0))
#   loss_ = loss_object(real, pred)

#   mask = tf.cast(mask, dtype=loss_.dtype)
#   loss_ *= mask

#   return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


# def accuracy(real, pred):
#   accuracies = tf.equal(real, tf.argmax(pred, axis=2))

#   mask = tf.math.logical_not(tf.math.equal(real, 0))
#   accuracies = tf.math.logical_and(mask, accuracies)

#   accuracies = tf.cast(accuracies, dtype=tf.float32)
#   mask = tf.cast(mask, dtype=tf.float32)
#   return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [36]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [37]:
tf.keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


# Если доступно несколько гпу
# расскоментируйте и сдвиньте остальной код ниже на 1 таб
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_ru])

#     learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
#         y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
# Обратите внимание на то как данные подаются в модель 
# помимо текста в модель еще нужно передать целевую последовательность
# но не полную а без 1 последнего элемента
# а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
# мы сравниваем выход из модели с целевой последовательностью уже с этим последним элементом

# сдвинутые последовательности создаются выше
# X_ru_dec - это переводной текст без последнего элемента
# X_ru_out - это переводной текст с последним элементом

model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=800,
             epochs=100,
             callbacks=[checkpoint]
             )
    
    
    

Epoch 1/100
  47/1188 [>.............................] - ETA: 9:28:23 - loss: 4.4469 - accuracy: 0.0467

(я не успела обучить трансформер, но думаю, что код ниже должен работать)

Задание:
Нужно какое-то время пообучать трансформер для машинного перевода (те же данные что и на семинаре) и, используя, получившуюся модель перевести какой-то достаточно длинный текст на английском языке (например, какую-то новость). При этом нужно переделать функцию translate так, чтобы она обрабатывала не каждый текст по отдельности, а сразу целиком (или батчами если вы возьмете совсем длинный текст). На предложения текст можно поделить внутри или снаружи функции.

In [ ]:
!pip3 install razdel

In [ ]:
from razdel import sentenize

In [ ]:
def translate(text):
    sents = list(sent.text for sent in sentenize(text))
    translated_sents = []
    for sent in sents:
        input_ids = encode(sent.lower(), tokenizer_en)

        input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                          [input_ids], maxlen=max_len_en, padding='post')

        output_ids = [tokenizer_ru.token_to_id('[CLS]') ]
        
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)
    
        
        while pred.numpy().argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[SEP]'), 
                                                                ]:
            if len(output_ids) > max_len_ru:
                break
            output_ids.append(pred.numpy().argmax(2)[0][-1])
            pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)

        translated_sents.append(' '.join([tokenizer_ru.id_to_token(i) for i in output_ids[1:]]))
    
    translated_text = ' '.join(translated_sents)

In [ ]:
text_to_translate = '''Почти год назад я переехал жить в Беларусь. Как вы уже догадались, если релокейт проходит без проблем, то такие истории на Хабр не пишутся. Однако, у меня есть что рассказать. Моя история будет не о славном граде Минске, белорусской идентичности, летних протестах и прочих, несомненно, важных вещах. Я расскажу вам об одной белорусской компании, в которой меня угораздило недолго поработать. История эта совершенно феерическая — что-то подобное могли бы снять режиссёры Silicon Valley. Всю свою карьеру я думал что так просто не бывает. Ну даже если и бывает — то я в это точно не вляпаюсь. Однако, я ошибался. Мне не повезло — переезд вышел мне боком и я влетел в целый водоворот событий. Они довольно поучительны и, как мне кажется, мой опыт будет интересен всем, кто планирует переезжать в ближайшем будущем. Меня зовут Павел и вот моя история.


Последние десять лет я жил в небольшом городке под Новосибирском и работал через UpWork. Помимо этого я писал на Хабр и вёл два своих публичных открытых проекта: Reinforced.Typings и Reinforced.Tecture. Денег на жизнь мне хватало и времени на свои проекты оставалось достаточно. Но однажды положение дел изменилось. Стартап, в котором я работал — вырос, руководство начало размываться новыми людьми. В реорганизованной компании мне не нашлось места, и меня уволили. Само собой, я страдал, лез на стены, орал от несправедливости и зарекался навек работать без equity. Но жизнь показала, что бывает и хуже. Гораздо хуже.


Я искал новую работу почти 4 месяца. Кроме удалёнки ничего не рассматривал. Однако, что-то упорно шло не по плану: на UpWork и до этого с C#/.NET было не густо, а в 2019 стало совсем печально. Мои пропозалы оставались без ответа, все свои коннекты я истратил. Remote на StackOverflow был только для обладателей паспорта США, а личные контакты и нетворкинг молчали. Моя финансовая подушка таяла, и скрепя сердце, я принял решение выйти из зоны комфорта и наконец-то релоцироваться из того медвежьего угла, в котором сидел последние 10 лет.


На Европы и Америки у меня не было ни денег ни связей. Работать спустя рукава в аутсорсе я решительно не хотел. Питер мне просто не нравится. В Москве дорого, пафосно, пробки и огромные расстояния. При этом цивилизации вокруг все равно хотелось. Ну стрёмно мне тратить 20 баксов и 3 часа на такси туда-обратно до ближайшего нормального бара. Ещё хотелось, чтобы IT-индустрия жила и шевелилась. Митапы всякие там, конференции, барные тусовки. Да и в целом — к 10+ годам индустриального опыта хочется уже работать с профессионалами своего дела, а не с недоучками.


Мои знакомые посоветовали Минск. Я знал, что в Республике Беларусь действует ПВТ и зарплаты весьма привлекательны. Более того — есть несколько примечательных продуктовых компаний: Viber, Stimulsoft, Pandadoc, Mapbox и TargetProcess имени Михаила Дубакова. В последней была открыта вакансия C#-разработчика и я отправил туда резюме. Я уже слышал про эту компанию — их продуктом пользовались мои хорошие знакомые из Новосиба. Компания быстро откликнулась на моё письмо, я прошел совершенно стандартное собеседование с тестовым заданием и, наконец, получил оффер.
Я перестраховщик, так что пособеседовался и в другие компании — это дало мне ещё один оффер. Пришлось долго и мучительно выбирать между двумя вариантами, и в итоге я выбрал TargetProcess. Его коллектив показался мне вполне нормальным и, что немаловажно, профессиональным. Гугл ничего плохого про компанию не выдавал — только радужные статьи на dev.by, где гордо рассказывается как компания справляется с процессом разработки без менеджеров и даже HR-ов. На собеседовании я тоже не заметил, что что-то может пойти не по плану.


Новосибирск — Минск

Оффер я принял и пообещал, что как штык буду на рабочем месте 10 декабря, в 11 утра. Но переезд дался мне нелегко. Обычно я не склонен к сантиментам, но тут было прямо не по себе. Деньги кончались, разработка своих проектов встала, а в Беларуси у меня нет решительно никого из знакомых. Я, блин, всё бросаю и еду в никуда. Срываюсь с насиженного места под влиянием обстоятельств. Всё, что у меня есть — это моя голова на плечах, технический опыт и вот этот оффер в инбоксе. Добираться предстоит совершенно одному. Случись что, помощи просить не у кого, вся надежда только на себя. Я старался себя успокоить тем, что TargetProcess вроде адекватная компания и уж точно не допустит какого-нибудь непотребства.


Сдал квартиру, доверил своего кота ответственому знакомому, выкинул ненужный хлам, что-то продал. Прекратил ходить на тренировки и сбрасывать вес, попрощался с немногочисленными друзьями. Единственное, что я себе оставил — автомобиль. Заднеприводный бизнес-кореец, продать который за вменяемую цену в Новосибирске мне не удалось. Сдавать своего коня перекупам по бросовой цене я категорически не желал. К чёрту. Занял денег у хороших людей и взял его с собой. Ближайший год мне всё равно не светило возвращаться (рекрутёр TargetProcess меня заверила что люди в этой компании работают подолгу — как раз то, что нужно).


Колёса пришлись весьма кстати. Я упаковал в багажник всё, что хотелось взять с собой, поставил на автовоз в Москву, а сам вылетел следом.


От Москвы до Минска надо было ехать своим ходом. Мне не впервой гонять на дальние расстояния по трассе, но и тут не без факапа. Автомобиль, заляпанный сантиметровым словом грязи, мне отгрузили с автовоза позже планируемого. Мойка, оплата, МКАД — и вместо расчётных 10 утра, я выехал чуть ли не в час дня. Про разницу в широтах и время захода солнца я, конечно же, был не в курсе. В общем, в 5 вечера прямо на трассе меня застала глухая ночь и дождь со снегом. Не самые удобные условия для вождения, но отступать уже некуда. Ехал наощупь, пытаясь не слететь с дороги.


А после пересечения Российско-Белорусской границы у меня отвалилась связь. Местную SIM-карту взять негде, пришлось вести по кэшированному куску яндекс.карт, указателям и километровым столбам. Часам к 11 вечера, наглухо вымотавшись, преодолев 4.5 тысячи километров за день, я наконец въехал в Минск. Вызвонил арендодателя (дай ей б-г здоровья!), с которой договорился заранее. Втащил вещи в квартиру и наконец грохнулся спать.


10 декабря, 11 утра. Я явился в офис и...


Хоп!

Проходит две недели. Я почти закрывая свою первую задачу (разгребание инфраструктурного легаси системы), только-только начинаю отходить от переезда, как вдруг...


меня уволили.
26 декабря 2019го года, под новогодние праздники я остался без работы.


Из денег у меня — зарплата за месяц (даже долги не вернёшь), вокруг — чужая страна, чужой город в предпраздничном ажиотаже. Проводить собеседования до 15 января никто не собирается, в городе введён режим "давай после праздников". До России — 600 километров, до дома вообще пол-Земли. Я даже не могу зарегистрироваться по месту пребывания — нужен трудовой договор — и любой встречный милиционер имеет полное право взять меня за жопу.


Никаких "профилактических бесед" со мной не проводили, претензий к моей работе не возникало, методических рекомендаций, что сделать лучше или по-другому — не поступало. Создалось ощущение, что компании я просто не упёрся. Технически это выглядело так: Андрей Хмылов — человек, который что-то вроде тимлида в команде отозвал меня в сторонку и сказал "мы решили не продолжать с тобой, потому что что-то мы не срабатываемся". На вопрос "А зачем вы вообще меня наняли?", я получил предельно чёткий ответ: "Просто мне показалось, что процессы в моей команде масштабируются на 6 человек, а не на 5".


Ну вы понимаете — человеку показалось.


Через 2 дня у меня состоялся диалог с техническим директором TargetProcess — Евгением Хасеневичем. И он был куда богаче на детали:


— В договоре чётко прописан испытательный срок, а проработал я 2 недели. Что значит "мы не срабатываемся"?
— По трудовому кодексу Беларуси мы имеем право уволить тебя на испытательном сроке когда захотим.
— Хорошо, я не эксперт в белорусском законодательстве. Но эм… я не в самом удачном положении — только переехал, у меня одни долги.
— Ну я не уточнил в бухгалтерии, можно ли заплатить тебе за 2 недели, поэтому компания — обрати внимание — закроет на это глаза и выплатит тебе месячную зарплату.
— То есть я ещё и спасибо сказать должен?
— Спасибо говорить не надо. Считай что это такая… добрая воля компании.
— Ещё раз: я одолжил больше денег, чтобы к вам приехать, чем то, что вы мне пишете в расчётном листке.
— А за что тебе платить? — собеседник расплылся в улыбке — Давай ты напишешь смету, я отнесу её в бухгалтерию, там всё посчитают.
— Я… э…
— Ну а что ты хотел? Переезд — это всегда риски, ты должен быть к ним готов.
— Так на дворе новогодние праздники. Мне придётся сидеть без работы месяц, потому что никто не собеседует.
— Ну если ты хороший программист, то в Минске-то без проблем найдёшь себе работу.
Рекрутёр TargetProcess порекомендовала мне не лезть на рожон. Скрепя сердце, я подписал доп. соглашение, что де-факто означало увольнение по собственному. Страна чужая, я только переехал — мне реально было стремно качать права.


Где-то тут пришло понимание, что соглашаться на релокейт "в ручном режиме" было довольно глупой затеей. Но издалека сотрудники TargetProcess не создавали впечатление безответственных мудаков, и мне чудилось, что я знаю на что иду. Пожалуй, надо завязывать с доверчивостью.


Я не переношу вранья и нарушения договорённостей. А здесь было целое комбо: внезапно выяснилось что никто мне ничего не обещал, а руководство продемонстрировало мне беспрецедентное хамство и выставило всё так, будто мне с барского преча позволили тут поработать. Человек, занимающий должность CTO, откровенно посмеялся мне в лицо. То есть меня, неплохого и опытного инженера, который в силу обстоятельств пошёл навстречу компании — облили говном и поставили в крайне неудобную ситуацию. Я считаю что во взрослом мире так дела не делаются. Раз компания в лице технического директора позволяет себе такое поведение, то мне ничего не мешает трактовать это инцидент как… ммм… дайте-ка посмотрю на свой банковский счёт… Точно! Кидалово на бабло.


Почему эта статья должна быть здесь?

В нашей индустрии есть ненадёжные люди. Мне кажется, что информация об их боевых заслугах обязана быть доступна по первому же поисковому запросу. Она должна светиться яркой неоновой вывеской. Чтобы любой желающий незамедлительно понял, кто есть кто, и на кого можно положиться в тонких вопросах переезда, а на кого нет. В случае с TargetProcess ситуация такова: поисковая выдача пестрит красивыми заголовками статей и интервью, описывающими радужно-конфетный мир. В объективной же реальности компания допускает крайне неэтичное отношение к релоцировавшимся сотрудникам. И я тому пример. Намеренно вводить людей в заблуждение — это очень и очень скверно.


Остаться без денег перед праздниками в чужой стране — это не та ситуация, в которую каждый из нас хочет попасть. Ладно, я — не голодающий Поволжья. Да и решать сложные проблемы мне не впервой. А если кто-то купится на оффер и приедет издалека с семьёй? С детьми? А если это будет молодой разработчик, который даже не подозревает о том, что так вообще бывает? Разумнее будет сократить риски, когда есть возможность и быть в курсе последствий.


Если бы эта статья была у меня-из-прошлого, я бы сэкономил кучу денег и нервов.


Жаль что опубликовать эту информацию раньше у меня не дошли руки. Просто они дрожали. И ещё глаз дёргался. Только сейчас я более-менее справился с тревожным расстройством и коробкой неврозов, которые мне достались всего после двух недель сотрудничества с героями статьи. Даже среди ночи уже не просыпаюсь. Почти. Ну что, вы всё ещё выгораете?


Тогда я вам расскажу как устроены стильные-модные-молодёжные компании.



Дело в том, что TargetProcess и её сотрудники свято чтят soft skills. Они очень любят рассказывать о себе. Например о том, как они обходятся без менеджеров и директора по управлению персоналом, какой у них стильный офис и как они получили ажно $5 млн инвестиций. Выходят хвалебные статьи, основатель активно визионирует, а члены самой-звёздной-команды выкладывают в командный инстаграмм фотографии с котлетами бабла. Короче, все при деле — "мы продуктовая компания, а не вот эти вот… ГАЛЕРЫ", "мы как белорусский Google!".


С самого начала у меня было какое-то внутреннее ощущение что дело тут нечисто. При таких замашках надо действительно быть стоящим игроком. Я насторожился, но в итоге списал всё на "должно быть я что-то не понимаю". Кажется, это называется эффект Даннинга-Крюгера. Но пост-фактум все странности становятся очевидны и я долго ругал себя за невнимательность.


Собеседование в TargetProcess квалификацию особо не проверяет, что явно не уровень продуктовой компании. Пусть так — у меня есть Github-аккаунт, у меня есть Хабр, у меня есть UpWork с отзывами, мою квалификацию можно проверить тысячью и одним способом, не задавая глупых вопросов про GetHashCode.


Чего действительно не ожидаешь от места, где "важны soft-skills" — собеседуют одни люди, а в команду попадаешь совершенно к другим. Уже это должно было стать красным флагом, но я, видимо, слишком наивен.


От момента, как я принял оффер до моего фактического приезда прошёл МЕСЯЦ. 30 дней. Четыре рабочих недели. Если компания декларирует упор на общение и софт-скиллы, то за эти 30 дней можно было организовать бесчисленное количество онлайн-встреч, посмотреть на меня через камеру, поговорить со мной текстом, голосом, познакомиться, выпить через skype в конце концов, дать тестовую задачу. Да господи, хоть онбординг провести и сказать мне по итогу — мол, нет, чувак, извини, не подходишь. Просто чтобы не дёргать меня через пол-страны и не жечь мои деньги. Но ничего из этого компания не сделала.


Кстати, про онбординг.


— Чем я могу помочь команде? Давайте для ознакомления я починю несколько застарелых багов, чтоли. Руки чешутся.
— Это так не работает — ответил мне человек, играющий в тимлида
— А как оно работает?
— Ну… не знаю… Я ожидаю некой автономности от своих сотрудников… Ну возьми разгреби воон ту штуковину.
Вот и весь тебе онбординг в устах Андрея Хмылова. Замечательный процесс, заточенный под быстрое и эффективное введение новых людей в работу над кодовой базой с более чем 15-летней историей. Особое внимание стоит обратить на заинтересованность, ответственность и готовность помочь.


Помогает в процессе онбординга полностью отсутствующая документация. Кусочек readme.md в репозитории, где-то документ в облаке, где-то заметка в личном блоге, где-то схема в онлайн-рисовалке, ссылка на которую передаётся из уст в уста. Передавать из уст в уста — наиболее общепринятый способ распространения информации о системе. Намеренно объяснять никто ничего не собирается — "ожидается автономность от сотрудников" (с).


Общего списка всех репозиториев, ссылок на них и объяснения, что в них есть — так же нигде нет. Системный администратор сделает вам пользователя чтобы вы могли залогиниться на свою рабочую машину — всё остальное в режиме "ну… попроси кого-нибудь". Вместо назначения ответственных за процесс принимается гениальное управленческое решение — закрыть информационные дыры с помощью soft-skills. Браво, узнаю управленческие практики google.


Какая ирония: компания, делающая инструмент для управления процессами разработки не может наладить процессы разработки сама у себя. Как это вообще блин работает?
Это очень похоже на job security driven development. Намеренно ограничивать знания о системе, чтобы никого не рискнули уволить и зарплату повышали не из-за профессиональных качеств, а потому что "никто же больше в этом не разберётся!". Страх, что "в системе больше никто не разберётся" довольно распространён в нашей сфере, но давайте замнём для ясности и коротко пройдёмся по самой системе чтобы понять, так ли всё плохо.


Система

Процессный и организационный бардак ни о чём не говорит при условии, что сама система сделана на совесть. Но


"Любая организация, проектирующая систему неизбежно создаст такую модель, которая будет повторять коммуникационную структуру самой организации".
— Закон Конвея.
Старик Конвей и здесь оказался прав.


Первое, что я увидел — единого кода системы не существует. Есть разные куски, написанные в разное время, разными людьми, с разными взглядами и разными убеждениями по вопросу "как надо". Разумеется, все они уже уволились, онбордить сотрудников никто не планирует, поэтому разбираться надо с нуля.


Если долго всматриваться в этот код, то создаётся впечатление, что авторы не проблему решали и не фичи делали, а показывали какие они умные. Что они знают паттерн strategy, или пробуют новый фреймворк за счёт работодателя, или новый язык, или просто креативят в пустоту. По итогу проблемы (зачастую выдуманные) решаются наименее очевидным из всех возможных способов. Технологические понты. Как и бывает в таких случаях, инструментарий авторы не понимали и не утруждали себя погружением в детали на предмет зачем это нужно, как оно работает и к месту ли. Что закономерно, ведь, как и было сказано, важны soft-skills. А стало быть на хрен идут проблемы продукта — тут надо показать коллегам какой ты умный и красивый. Вершина и кульминация этого буйства сознания — использование самописной монады Maybe<>, которая где-то в середине стека вызовов прагматично разворачивается в if (maybe.HasValue). Функциональное программирование вам, так сказать, в production.


Выстроив в голове модель сущностей, открываешь для себя другую особенность. Называется "без штанов, но в шляпе". Приведу абстрактный пример: вообразите себе витиеватые заросли Repository Pattern, всё по науке, интерфейс-реализация, в разных неймспейсах, с заделом на тестирование. Вот вся эта красота развешена поверх… статического подключения к БД! Такое нам в Новосибирск из Индии везут на рефакторинг! Тоннами! Я ещё будучи джуном понял, почему подключение к БД нельзя пихать в статический контекст. Что же помешало "лучшим умам" не наступить на эти грабли? Видимо, тот факт что про грамотное управление лайфтаймом подключения не расскажешь на тим-митинге и внутренней конфе. А вот прочитать статью на википедии и красиво всё разложить по репозиториям — вполне себе социально одобряемое. Десять soft skills из десяти.


Обычно подобный треш и угар в системе пресекается системным архитектором. Но это не наш случай: в дополнение к soft-skills, тут полный agile. То есть предполагается, что все сотрудники — профессиональные инженеры, которые сами могут договориться и принять правильное техническое решение. Вкусно, как Orbit со вкусом design by committee.


Видавший виды руководитель знает, что agile и делегирование принятия технических решений команде на практике означает "слабоумие и отвага", если не проводить жесточайшего кадрового отбора по хард-скиллам. Но чтобы организовать такой отбор во-первых нужен человек неприлично высокой квалификации, который и будет проводить собеседование, а во-вторых — опытный управленец с намётанным глазом.


В рамках своей первой и единственной задачи я разгребал код, написаный местным техническим директором. Что ж… если человек не может спроектировать простое консольное приложение, принимающее флаги и делающее действия, то строгий кадровый отбор по хард-скиллам — явно история не про него. Вот и остаётся писать в буклетах "процесс разработки не нуждается в менеджерах" и загадочно улыбаться.


Сам по себе отвратительный код — это нормально. Для какого-нибудь аутсорса. Разница в том, что аутсорс-компания обычно не претендует на лавры best place for work и какой-то особый уровень экспертизы. Да и лица там не настолько высоконагружены, чего греха таить.


Кстати, о лицах

Вообще я не очень хорошо разбираюсь в людях, предпочитаю всё-таки системы. Но тут сам б-г велел, ведь радужно-оптимистичные статьи жанра "личностный рост для разработчика" предписывают работать с талантливыми людьми. Сам не читал, но что-то такое слышал на краю Интернета. Грех не воспользоваться случаем.


И вот впервые в жизни я увидел молодых мальчиков и девочек с высокодуховными лицами, лоснящихся от высоких зарплат. Они не ходят, они как будто парят над землёй, стоя на облаке из квалификации. Ну, думаю, наконец-то. Вот они — настоящие инженеры. Сбылась мечта идиота, я работаю не с аутсорс-недоучками и вайтишниками, а с самыми, что ни на есть мозгами. Которые в курсе трендов и технологий!


Я обратился к людям из "самой звёздной команды"(tm) в попытке поговорить с ними о технических штуках, рассказать о том, через что сам прошёл, обсудить тенденции, архитектуры разных вещей (в том числе и самой системы)...


Видимо, что-то пошло не так. В ответ я получил пачку и без того мне известных buzzword-ов, одухотворённо-покровительственный взгляд и отшучивание. Всю первую неделю я гадал — что же не так? Может я как-то… не знаю… Вопросы не те задаю? Или, может, надо не про себя рассказывать, а больше вопросов задавать, попросить научить меня чему-нибудь?


Всё оказалось гораздо проще. Технические дискуссии коллег не интересуют. Поначалу казалось что их вообще мало что интересует, но потом я просёк фишку. "Высоконагруженные лица" оживляются на разговорах о чём-то более мирском. Ну вы знаете, не об этих ваших абстракциях, паттернах и технологиях, а о чём по-проще: кто в каком ресторане обедал, где провести тимбилдинг, куда поехать в отпуск, кто какой гаджет купил и прочие темы "за жизнь" "за покупку второй бэхи".


Тут у меня, наверное, нет комментариев. Видимо вот такой он, градус дискуссии профессионалов высокого класса. И ничего с этим не поделаешь.


Итог

Вот что мне всю жизнь непонятно — почему инвесторы вкладываются в такие компании? Предположим, что у меня были бы деньги на долю и я хочу разобраться: а что тут, собственно, покупать? Обычно в стартапах покупают рост в надежде, что он будет взрывной. Но взрывной рост сложно организовать без продукта, попадающего в голубой океан пользовательских потребностей. Здесь требуются удачливые визионеры, грамотные маркетологи, Product Manager-ы и работающий как часы продакшен.


В TargetProcess: единственный визионер (он же основатель, он же Миша Дубаков) ушёл с продукта, плотняком ударился в идею околокорпоративного no-code. TargetProcess после этого начал заниматься хаотичным метанием в надежде догнать Jira под руководством каких-то сомнительных личностей.


Продакшен? Увы, поверхностный аудит кода и разговор с людьми отчётливо показал что продакшен больше интересует красивая жизнь, нежели продукт или технологии.


Взрывной рост сложно пережить без чёткого управления и подготовленных процессов. Здесь уровень разгильдяйства вкупе с возрастом предприятия даёт явное понимание что людей, способных внедрить и настроить процессы в компании нет и никогда не было. Документация, онбординг, ответственные, передача информации? По всем пунктам провал. Взрывной рост разорвёт всю конструкцию на куски по наложенным на коленке швам.


Однако TargetProcess не уникален. Подобных компаний на рынке полно. Думаю, будет не лишним написать пару советов как не угодить в местечковый Theranos.


Как не наступить на мои грабли

Я детально переварил свой опыт, поговорил со знакомыми из кадрового бизнеса, расспросил успешно релоцировавшихся как прошло у них, почитал интернет. На основе полученной информации я составил свой список основных моментов, которые необходимо учитывать при релокации. Как видите, откровенный трешак, маскирующийся под серьёзный бизнес на рынке присутствует, поэтому надо держать ухо востро, чтобы не оказаться в жопе. Мне видится так, что эта информация, написанная моими нервами, деньгами и психическим здоровьем будет полезной для желающих переехать. Да и в целом держите это в голове при любой смене места работы.


Первое, что надо помнить при релокейте — вы в заведомо более рисковом положении, нежели компания. У вас меньше и денег, и времени, и нервов. И правовое поле не ваше, и территория чужая. Вы в максимально уязвимом положении. Кинуть вас и оставить ни с чем — проще, чем отобрать леденец у ребёнка. Поэтому надо выбивать из принимающей стороны максимальные бонусы. Это не "программисты зажрались", а необходимая мера страховки для обеих сторон: вы худо-бедно приобретаете уверенность, что даже если что-то пойдёт не так — вы не окажетесь один на один с полной жопой. Компания вкладывая деньги в ваш переезд, верифицирует управленческое решение и страхует себя от негативных отзывов в будущем. Короче, даже не начинайте диалог о релокации без обсуждения релокейшн-пакета и даже не пытайтесь обеспечить его своими руками, если компания не предлагает. Серьёзно, просто не надо.


В рамках релокейшн-пакета требуйте:


Полную или частичную оплату переезда. Как минимум билет в одну сторону вам должны оплатить. Совсем прекрасно если вас встретят в аэропорту или куда вы там прибываете;
Временного размещения. Требуйте предоставления корпоративной квартиры или отеля на период, пока вы не найдёте постоянное жильё. Совсем хорошо если компания помогает с поисками;
Relocation bonus. Чтобы к первым вашим зарплатам была прибавка на основе тот факта, что вы переехали. Обычно у переехавших с деньгами не очень, поэтому помощь компании необходима. Совсем хорошо если бонус единовременный и выдаётся сразу;
Серебряный парашют. Редко, но такое бывает. Это формальное обязательство выплатить вам компенсацию (обычно в размере нескольких зарплат) в случае увольнения в первые 1-2-6-12 месяцев (как договоритесь);
Оформления приглашений, виз (в том числе для членов семьи), страховок и прочих въездных и не очень документов. В идеале — без вашего участия.

Удостоверьтесь заранее:


Что google, GlassDoor, любые сайты с отзывами не выдают о компании ничего плохого. Если есть хоть один (один!) негативный отзыв — ищите другой вариант. Поверьте, это не та рулетка, в которую вам стоит играть;
Что в целевой компании есть отработанный процесс онбординга новых сотрудников;
Что трудовой договор не подразумевает расторжения ранее определённого срока (так же именуемого испытательным);
Что компания ранее уже занималась трудоустройством релоцирующихся сотрудников. Это чтобы в самый неподходящий момент кадровики и бухгалтеры не начали судорожно звонить в миграционную службу и узнавать какой пакет документов, в какие сроки и куда надо предоставить. Совсем хорошо если у компании есть опыт релокации именно из вашей страны;
Что вы будете работать именно с теми людьми, которые вас собеседовали;
Что мотивация вашего найма — не "абы взять" на потеху инвестора, а необходимость решения конкретных проблем нанимающей стороны. Спрашивайте явно на собеседовании — "зачем вы меня нанимаете? какая у вас конкретно боль?";
Что у компании есть чёткая и понятная всем сторонам политика увольнений.

Вообще, про увольнения, конечно, я рекомендую прямо сейчас поговорить с вашим руководством. Поймайте своего непосредственного начальника за пуговицу и добейтесь ответа на вопрос "при каких условиях вы меня уволите"? Один мой хороший знакомый предложил отличную формулировку: "какое моё поведение приведёт к вашему решению о моём увольнении?". Тут самое страшное — даже не жёсткость условий, а определённость, с которой вам дали ответ. Помните — если руководитель не понимает за что людей надо увольнять, а за что — нет, то вы будете уволены в самый неожиданный момент по самой неожиданной причине. Совсем хорошо будет сопоставить ответ на этот вопрос с трудовым законодательством целевой страны. Если политики увольнения в компании нет, то это легитимная причина для пересмотра вашей зарплаты в сторону повышения в связи с необходимостью покрытия рисков. Или смены работы на место, где управлять умеют немного лучше.


Послесловие

Я хочу поблагодарить ребят из подкаста "Мы обречены" и Филу лично за то, что не остались глухи к моей ситуации и сильно помогли мне успокоиться, взять себя в руки и написать слаженный текст о произошедшем. Спасибо, ребята, без вас я бы не справился — совсем одному переживать такое было бы крайне тяжело. Эта история слишком сильно меня подкосила и выбила из колеи, но с вашей помощью я выгреб.


В феврале я нашёл другую работу и мои нынешние коллеги, должность и зарплата меня очень радуют.


Потом случился коронакризис. А потом, летом в Беларуси произошли всем известные события. Не смотря на подпорченное первое впечатление, мне посчастливилось встретить очень много совершенно прекрасных людей, поддержать их в меру возможностей и посодействовать им. Беларусь прекрасна, белорусы абсолютно замечательные. Но, как и в любом обществе, здесь так же есть и хитрые шарлатаны, и безответственные дураки, которым нельзя доверять и проходимцы от IT-индустрии, возомнившие себя мессиями. Я категорически не желаю чтобы персонально мой негативный опыт и конкретно TargetProcess бросал тень на всё белорусское IT. Тот факт, что лично мне попалась чудовищно токсичная компания с непрофессиональным руководством, вовсе не означает, что везде так. Нет.


В Беларуси полно замечательных кадров, интересных компаний, умных и достойных людей. Видимо, я просто крайне неудачно выбрал представителей индустрии для начала работы.


Берегите себя.


P.S: совсем забыл подсветить ещё один маленький, но интересный момент. Сразу после инцидента вакансия разработчика на сайте TargetProcess поменялась с Senior на Junior. А через некоторое время и вовсе исчезла. Вот теперь всё.'''

In [ ]:
translate(text_to_translate)